In [50]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from utils import seed_everything, read_yaml_conf, write_yaml_conf, ROOT_DIR
from data.data_prep_main import prepareData
from model.train_main import train_model
from model.evaluate import evaluate_nn_model_list, save_config_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import torch
import multiprocessing

# Parameters

In [34]:
#Experiment name
experiment = '1'

In [29]:
#Device data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cpu = multiprocessing.cpu_count()

In [54]:
#From conf file
conf = read_yaml_conf('../src/conf.yaml')

path_data = conf['paths']['path_data']
path_output = conf['paths']['path_output']
path_model = conf['paths']['path_model']

normalization_type = conf['paths']['path_data']
use_train_test_for_norm = conf['paths']['path_data']

#KDE features
add_kernels = conf['features']['add_kernels']
use_log_for_kernel_diff = conf['features']['use_log_for_kernel_diff']
inverse_kde = conf['features']['inverse_kde']
ratio_inverse_kde = conf['features']['ratio_inverse_kde']
use_diff_kde = conf['features']['use_diff_kde']
exclude_c_from_kde = conf['features']['exclude_c_from_kde']
exclude_g_from_kde = conf['features']['exclude_g_from_kde']

#Dimension reduction
g_removal_count = conf['features']['g_removal_count']
perform_pca = conf['features']['perform_pca']
pca_for_c = conf['features']['pca_for_c']
pca_for_kde = conf['features']['pca_for_kde']

add_c_stats = conf['features']['add_c_stats']


#Oversampling
augment_data = conf['augmentation']['augment_data']
additional = conf['augmentation']['additional']
granularity = conf['augmentation']['granularity']
max_dev = conf['augmentation']['max_dev']
normal_std_dev = conf['augmentation']['normal_std_dev']

#Add control data
control_share_in_train = conf['augmentation']['control_share_in_train']
add_control_from_test = conf['augmentation']['add_control_from_test']



#Holdout
validation_ratio = conf['model']['validation_ratio']

#Cross validation 
folds = conf['model']['folds']
fold_type = conf['model']['fold_type']
oof_type = conf['model']['oof_type']
oof_threshold = conf['model']['oof_threshold']

n_seeds = conf['model']['n_seeds']

label_smoothing = conf['model']['label_smoothing']
oof_loss_limit = conf['model']['oof_loss_limit']
nn_architecture = conf['model']['nn_architecture']


model_nonscored = conf['post_processing']['model_nonscored']

# Run

In [31]:
prepared_data = prepareData(path_data,validation_ratio=validation_ratio,folds=folds, cpu=cpu, normalization_type=normalization_type,
                            g_removal_count=g_removal_count,exclude_c_from_kde=exclude_c_from_kde,exclude_g_from_kde=exclude_g_from_kde,add_c_stats=add_c_stats,
                            add_kernels=add_kernels, use_diff_kde=use_diff_kde, use_train_test_for_norm=use_train_test_for_norm,
                            perform_pca=perform_pca, pca_variance_threshold=0.95,pca_for_kde=pca_for_kde,pca_for_c=pca_for_c,
                            use_log_for_kernel_diff=use_log_for_kernel_diff, inverse_kde=inverse_kde, ratio_inverse_kde=ratio_inverse_kde,
                            granularity=granularity,max_dev=max_dev,normal_std_dev=normal_std_dev,additional=additional
                            )

import data
transform cat features
kde kernels calculations
remove g columns with low variation
kde features
normalize features


In [35]:
nn_model_list = train_model(prepared_data, conf, ROOT_DIR+'/models/experiment_'+experiment+'_')



START WITH SEED: 0
Start training fold 1 of 3
Add 0 control to training set, representing 0% of vehicle size
Epoch 1/1 	 train loss: 0.02197 	 val loss: 0.01925 	 time: 34s 	 lr: 0.002507 	 
Start training fold 2 of 3
Add 0 control to training set, representing 0% of vehicle size
Epoch 1/1 	 train loss: 0.02199 	 val loss: 0.01944 	 time: 34s 	 lr: 0.002507 	 
Start training fold 3 of 3
Add 0 control to training set, representing 0% of vehicle size
Epoch 1/1 	 train loss: 0.02179 	 val loss: 0.01907 	 time: 40s 	 lr: 0.002507 	 
Saving models


In [39]:
results_mean = evaluate_nn_model_list(prepared_data, nn_model_list,ls_floor=1e-5,ls_ceil=1e-5,activation_threshold=0.3,floor=1e-7)

oof: 0.017834 	 oof post: 0.017560 	 oof ls: 0.017422 	 oof excl: 0.017422 	 holdout: 0.018051 	 holdout post: 0.017768 	 holdout ls: 0.017588 	 holdout excl: 0.017588


In [60]:
save_config_score(conf, results_mean, ROOT_DIR + '/models/conf_experiement_'+experiment+'.yaml')